# Reading Order Detection

In [ ]:
# Requirements for the custom doctr model
# Restart runtime after the cell is executed
! pip install 'python-doctr[torch]==0.5.1'
!pip install rapidfuzz==2.6.0
! pip install tensorflow_addons
! pip install tf2onnx

In [ ]:
import os
os.environ['USE_TORCH'] = '1'

In [ ]:
# Import the necessary libraires
import torch
import os
from doctr.models import ocr_predictor
from collections import OrderedDict
from doctr.io import DocumentFile
import cv2
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import ast
import matplotlib.pyplot as plt
import math
from scipy.stats import gaussian_kde
import networkx as nx
from scipy.spatial.distance import euclidean

## Helper Functions

In [ ]:
# Function to calculate the euclidean distance between 2 points
def euclidean_distance(point1, point2):
    squared_diff = (point1 - point2) ** 2
    sum_squared_diff = np.sum(squared_diff)
    distance = np.sqrt(sum_squared_diff)

    return distance

In [ ]:
# Function to extract integer coordinates read as strings while reading dataframes from .csv files
def parse_string(string, start_char, end_char):
    start_index = string.index(start_char) + 1
    end_index = string.index(end_char)
    parsed_string = string[start_index:end_index]
    return parsed_string

In [ ]:
# Function to calculate the euclidean distance between 2 points when not in the form of a list
def euclidean_distance1(coord1, coord2):
    point1 = np.array(coord1)
    point2 = np.array(coord2)
    squared_diff = (point1 - point2) ** 2
    sum_squared_diff = np.sum(squared_diff)
    distance = np.sqrt(sum_squared_diff)
    return distance

## Stage-1: Bounding boxes B = B1, B2, ... for individual document words are obtained using a pre-trained text detection model.

In [ ]:
# Function for doctr predictions. Returns (x,y) coordinates of the top-left and bottom-right corner of each box.
def doctr_predictions(directory):

    doc = DocumentFile.from_images(directory)
    result = predictor(doc)
    dic = result.export()

    page_dims = [page['dimensions'] for page in dic['pages']]

    regions = []
    abs_coords = []

    regions = [[word for block in page['blocks'] for line in block['lines'] for word in line['words']] for page in dic['pages']]
    abs_coords = [
    [[int(round(word['geometry'][0][0] * dims[1])),
      int(round(word['geometry'][0][1] * dims[0])),
      int(round(word['geometry'][1][0] * dims[1])),
      int(round(word['geometry'][1][1] * dims[0]))] for word in words]
    for words, dims in zip(regions, page_dims)
    ]

    return abs_coords

In [ ]:
# Upload the db_resnet50.pt model
# The cell might throw a Runtime error if run on Google colab due to CUDA installation issues
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictor = ocr_predictor(pretrained=True)

state_dict = torch.load("/content/db_resnet50.pt")

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]
    new_state_dict[name] = v


predictor.det_predictor.model.load_state_dict(new_state_dict)

## Stage-2: The spatial proximity relationships among the bounding boxes B are used to construct a disconnected graph G with the individual boxes as graph nodes. The subgraphs Gs = g1, g2,... of G constitute the set of paragraphs

### Code snipped to generate KDE thresholds

In [ ]:
# Function to get Gaussian KDE
kernel_bandwidth = 0.1 # Standard value

def kde_estimate(data):
  kde = gaussian_kde(data, bw_method=kernel_bandwidth)
  x = np.linspace(min(data), max(data), 1000)
  kde_values = kde.evaluate(x)
  peak_index = np.argmax(kde_values)
  peak_value = x[peak_index]
  return math.ceil(peak_value)


In [ ]:
# Function to get the closest vertical boxes to be fed to the KDE for vertical thresholds
# Input: Dataframe with word bounding boxes
# Output: Average vertical distance for each box as a list
# Use 8 closest neighbours (Horizontal neighbours - 6 and vertical neighbours -2)
def find_closest_neighbors(df):
    horizontal = []
    vertical = []
    for index, row in df.iterrows():
        current_box = row[['Top', 'Bottom', 'Left', 'Right']].values
        distances_horizontal = []
        distances_vertical = []
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_box = other_row[['Top', 'Bottom', 'Left', 'Right']].values
                distance_left_to_right = euclidean_distance1(current_box[2], other_box[3])
                distance_right_to_left = euclidean_distance1(current_box[3], other_box[2])
                distances_horizontal.extend([distance_left_to_right, distance_right_to_left])
        distances_horizontal.sort()
        s = sum(distances_horizontal[:6])
        a = s/6
        horizontal.append(a) # Average horizontal distance per box stored as a list

        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_box = other_row[['Top', 'Bottom', 'Left', 'Right']].values
                distance_top_to_bottom = euclidean_distance1(current_box[1], other_box[0])
                distance_bottom_to_top = euclidean_distance1(current_box[0], other_box[1])
                distances_vertical.extend([distance_top_to_bottom, distance_bottom_to_top])
        distances_vertical.sort()
        v = sum(distances_vertical[:2])
        t = v/2
        vertical.append(t) # Average vertical distance per box stored as a list

    return horizontal, vertical

### Code snippets to build the graph

In [ ]:
# Function to calculate the centers of all the 4 edges of the bounding boxes and stored in a dataframe as the Top, Bottom, Left and Right.
# An unique ID is assigned to each box.
# input: Dataframe with bounding box coordinates, new dataframe to which information of the edge centers are appended.
def calculate_center_points(df, new_df):
    top = []
    bottom = []
    left = []
    right = []
    id = []

    for i in range(0,len(df)):
      x1,y1,x2,y2 = df[0][i][0], df[0][i][1], df[0][i][2], df[0][i][3] # Retriving the x and y coordinates of the top-left & bottom-right corners of the boxes.
      # (x1,y1) = top-left coordinates
      # (x2,y2) = bottom-right coordinates

      # Calculating centers of the edges of the bounding boxes
      top_center = [(x1 + x2) / 2, y1]
      bottom_center = [(x1 + x2) / 2, y2]
      left_center = [x1, (y1 + y2) / 2]
      right_center = [x2, (y1 + y2) / 2]
      top.append(top_center)
      bottom.append(bottom_center)
      left.append(left_center)
      right.append(right_center)
      id.append(i)
    new_df['Top'] = top # Center (x,y) coordinates for the top edge of the bounding box
    new_df['Bottom'] = bottom # Center (x,y) coordinates for the bottom edge of the bounding box
    new_df['Left'] = left # Center (x,y) coordinates for the left edge of the bounding box
    new_df['Right'] = right # Center (x,y) coordinates for the right edge of the bounding box
    new_df['Id'] = id # Unique ID assigned to each box

In [ ]:
# Function to calculate the ID of the box to the right of the current box.
# Function calculates the immediate right boxes of all the bounding boxes in the page and appends it to the dataframe.
# Input: Dataframe with edge centers, horizontal threshold from the KDE function
# Information regarding the rightbox is appended directly to the dataframe in the form of [distance to the right box, ID of the right box].
# Default value: [-1,0]
def calculate_rightbox(new_df,x):
    # new_df = dataframe containing information, x = horizontal threshold
    right = []
    for i in range(len(new_df)): # Traversal to calculate the immediate right box for every box in the dataframe
        dist = []
        id = []
        for j in range(len(new_df)): # Traverse every other bounding box to find the closest box within the horizontal threshold
        # Calculate euclidean distance between the right center of the current box & the left center of every other box & append all such box ids within the threshold to a list.
            distance = euclidean_distance(np.array(new_df['Left'][j]), np.array(new_df['Right'][i]))
            y_distance = abs(new_df['Right'][i][1] - new_df['Left'][j][1])
            if 0 <= distance < x and i != j and y_distance < 20: # 20 pixels to eliminate boxes which fell with the x threshold but belonged to different lines. Both boxes had to be part of the same line.
                dist.append(distance)
                id.append(j)
        if dist:
            t = np.argmin(dist) # Calculate the closest box from the list and append the distance and the ID to the dataframe.
            right.append([np.min(dist), id[t]])
        else:
            right.append([-1, 0])

    new_df['Right_Box'] = right

In [ ]:
# Function to calculate the ID of the box to the left of the current box.
# Function calculates the immediate left boxes of all the bounding boxes in the page and appends it to the dataframe.
# Input: Dataframe with edge centers, horizontal threshold from the KDE function.
# Information regarding the leftbox is appended directly to the dataframe in the form of [distance to the left box, ID of the left box].
# Default value: [-1,0]
def calculate_leftbox(new_df,x):
    # new_df = dataframe containing information, x = horizontal threshold
    left = []
    for i in range(len(new_df)): # Traversal to calculate the immediate left box for every box in the dataframe
        dist = []
        id = []
        for j in range(len(new_df)):  # Traverse every other bounding box to find the closest box within the horizontal threshold
        # Calculate euclidean distance between the left center of the current box & the right center of every other box & append all such box ids within the threshold to a list.
            distance = euclidean_distance(np.array(new_df['Right'][j]), np.array(new_df['Left'][i]))
            y_distance = abs(new_df['Left'][i][1] - new_df['Right'][j][1])
            if 0 <= distance < x and i != j and y_distance < 20: # 20 pixels to eliminate boxes which fell with the x threshold but belonged to different lines. Both boxes had to be part of the same line.
                dist.append(distance)
                id.append(j)
        if dist:
            t = np.argmin(dist) # Calculate the closest box from the list and append the distance and the ID to the dataframe.
            left.append([np.min(dist), id[t]])
        else:
            left.append([-1, 0])

    new_df['Left_Box'] = left

In [ ]:
# Function to calculate the ID of the box to the top of the current box.
# Function calculates the immediate top boxes of all the bounding boxes in the page and appends it to the dataframe.
# Input: Dataframe with edge centers, vertical threshold from the KDE function
# Information regarding the topbox is appended directly to the dataframe in the form of [distance to the top box, ID of the top box].
# Default value: [-1,0]
def calculate_topbox(new_df,y):
    # new_df = dataframe containing information, y = vertical threshold
    top = []
    for i in range(len(new_df)): # Traversal to calculate the immediate top box for every box in the dataframe
        dist = []
        id = []
        for j in range(len(new_df)): # Traverse every other bounding box to find the closest box within the vertical threshold
        # Calculate euclidean distance between the top center of the current box & the bottom center of every other box & append all such box ids within the threshold to a list.
            distance = euclidean_distance(np.array(new_df['Bottom'][j]), np.array(new_df['Top'][i]))
            if 0 <= distance < y and i != j:
                dist.append(distance)
                id.append(j)
        if dist:
            t = np.argmin(dist) # Calculate the closest box from the list and append the distance and the ID to the dataframe.
            top.append([np.min(dist), id[t]])
        else:
            top.append([-1, 0])

    new_df['Top_Box'] = top

In [ ]:
# Function to calculate the ID of the box to the bottom of the current box.
# Function calculates the immediate bottom boxes of all the bounding boxes in the page and appends it to the dataframe.
# Input: Dataframe with edge centers, vertical threshold from the KDE function.
# Information regarding the bottombox is appended directly to the dataframe in the form of [distance to the bottom box, ID of the bottom box].
# Default value: [-1,0]
def calculate_bottombox(new_df,y):
    # new_df = dataframe containing information, y = vertical threshold
    bottom = []
    for i in range(len(new_df)): # Traversal to calculate the immediate bottom box for every box in the dataframe
        dist = []
        id = []
        for j in range(len(new_df)): # Traverse every other bounding box to find the closest box within the vertical threshold
        # Calculate euclidean distance between the bottom center of the current box & the top center of every other box & append all such box ids within the threshold to a list.
            distance = euclidean_distance(np.array(new_df['Top'][j]), np.array(new_df['Bottom'][i]))
            if 0 <= distance < y and i != j:
                dist.append(distance)
                id.append(j)
        if dist:
            t = np.argmin(dist) # Calculate the closest box from the list and append the distance and the ID to the dataframe.
            bottom.append([np.min(dist), id[t]])
        else:
            bottom.append([-1, 0])

    new_df['Bottom_Box'] = bottom

In [ ]:
# Function to generate an image of the page with the bounding boxes as nodes and the connections as edges between the boxes.
# Input: Image read using CV2
# Output: Image of page with bounding boxes and connections between them.
def make_connections(image):

  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  image_with_boxes = image_rgb.copy()

  for index, row in euclidean.iterrows():
      left = int(row['Left'][0])
      right = int(row['Right'][0])
      top = int(row['Top'][1])
      bottom = int(row['Bottom'][1])
      box_id = int(row['Id'])

      width = right - left
      height = bottom - top

      top_left = (left, top) # Recalculating the top-left and bottom-right to plot the bounding boxes
      bottom_right = (right, bottom)

      cv2.rectangle(image_with_boxes, top_left, bottom_right, (255, 0, 0), 4)

      # Uncomment these lines to show the unique IDs given to each box.
      # label_position = (left, top - 10)
      # cv2.putText(image_with_boxes, str(box_id), label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

      top_adjacent_id = int(row['Top_Box'][1])
      bottom_adjacent_id = int(row['Bottom_Box'][1])
      left_adjacent_id = int(row['Left_Box'][1])
      right_adjacent_id = int(row['Right_Box'][1])

      if top_adjacent_id != 0: # Draw the connection between current box and top box
          top_adjacent_row = euclidean[euclidean['Id'] == top_adjacent_id].iloc[0]
          top_adjacent_center = int(top_adjacent_row['Bottom'][0]) , int(top_adjacent_row['Bottom'][1])
          cv2.line(image_with_boxes, (int(left) + width // 2, int(top)), top_adjacent_center, (0, 255, 0), 4)

      if bottom_adjacent_id != 0: # Draw the connection between current box and bottom box
          bottom_adjacent_row = euclidean[euclidean['Id'] == bottom_adjacent_id].iloc[0]
          bottom_adjacent_center = int(bottom_adjacent_row['Top'][0]) , int(bottom_adjacent_row['Top'][1])
          cv2.line(image_with_boxes, (int(left) + width // 2, int(bottom)), (int(bottom_adjacent_center[0]), int(bottom_adjacent_center[1])), (0, 255, 0), 4)

      if left_adjacent_id != 0: # Draw the connection between current box and left box
          left_adjacent_row = euclidean[euclidean['Id'] == left_adjacent_id].iloc[0]
          left_adjacent_center = int(left_adjacent_row['Right'][0]) , int(left_adjacent_row['Right'][1])
          cv2.line(image_with_boxes, (int(left), int(top) + height // 2), (int(left_adjacent_center[0]), int(left_adjacent_center[1])), (0, 255, 0), 4)

      if right_adjacent_id != 0: # Draw the connection between current box and right box
          right_adjacent_row = euclidean[euclidean['Id'] == right_adjacent_id].iloc[0]
          right_adjacent_center = int(right_adjacent_row['Left'][0]) , int(right_adjacent_row['Left'][1])
          cv2.line(image_with_boxes, (int(right), int(top) + height // 2), (int(right_adjacent_center[0]), int(right_adjacent_center[1])), (0, 255, 0), 4)

  return image_with_boxes



## Stage-3: Using the spatial extent information of word bounding boxes that constitute each paragraph subgraph, the set of subgraphs are processed to obtain a permutation sequence of g1,g2, ... This sequence represents the sequential order of paragraphs in the document.

In [ ]:
# Function to convert dataframe into graphs using networkx
# Input: Dataframe containing bounding boxes
# Output: Return Grap
def create_graphs(euclidean):
  G = nx.Graph()
  for _, row in euclidean.iterrows():
      box_id = row['Id']
      right_box = row['Right_Box']
      left_box = row['Left_Box']
      top_box = row['Top_Box']
      bottom_box = row['Bottom_Box']
      # Indexing the box IDs
      right_box_id = parse_string(right_box," ","]")
      left_box_id = parse_string(left_box," ","]")
      top_box_id = parse_string(top_box," ","]")
      bottom_box_id = parse_string(bottom_box," ","]")
      # Adding the nodes to the graph and edges
      G.add_node(box_id)
      if right_box[0] != -1 and right_box[1] != "-":
          G.add_edge(int(box_id), int(right_box_id))
      if left_box[0] != -1 and left_box[1] != "-":
          G.add_edge(int(box_id), int(left_box_id))
      if top_box[0] != -1 and top_box[1] != "-":
          G.add_edge(int(box_id), int(top_box_id))
      if bottom_box[0] != -1 and bottom_box[1] != "-":
          G.add_edge(int(box_id), int(bottom_box_id))

  # Store all the node in a variable
  connected_components = nx.connected_components(G)

  # Generate subgraphs using DFS
  subgraphs = [G.subgraph(component) for component in connected_components]

  num_subgraphs = len(subgraphs)
  num_cols = 2
  num_rows = (num_subgraphs + num_cols - 1) // num_cols
  fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 8))
  axs = axs.flatten()

  # Code draw the subgraphs
  for i, subgraph in enumerate(subgraphs):
      ax = axs[i]
      pos = nx.spring_layout(subgraph, k=0.5)
      nx.draw(subgraph, pos, with_labels=True, node_color='lightblue', node_size=100, font_size=8, edge_color='gray', ax=ax)
      ax.set_title(f'Subgraph {i+1}')


  if num_subgraphs < len(axs):
      for j in range(num_subgraphs, len(axs)):
          fig.delaxes(axs[j])

  plt.tight_layout()
  plt.show()

  return G



In [ ]:
# Function to generate a paragraph.json with all the words within each paragraph as separate components
def get_paras(G):
  df = pd.DataFrame()
  connected_components = list(nx.connected_components(G))
  connected_components_list = [list(component) for component in connected_components]
  data = {"connected_components": connected_components_list}
  json_data = json.dumps(data)
  file_path = "paragraph.json" # Write it to paragraph.json

  with open(file_path, "w") as json_file:
      json_file.write(json_data)

In [ ]:
# Function to create a dataframe for paragraphs, assign a unique ID for each paragraph and generate images with paragraph bounding boxes.
# Input: image, target component read from paragraph.json file, Dataframe(euclidean) with all the bounding boxes and image_filename.
# Output: Dataframe with paragraph information
def recognise_paragraphs(image, target_components, euclidean, image_filename):

 component = pd.DataFrame()
 count = 0
  #  Uncomment the lines to generate images with paragraph bounding boxes
  #  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #  Create a copy of the image to draw the boxes and labels on
  #  image_with_boxes = image_rgb.copy()

 for i in target_components: # Traverse through all the components
    left = []
    right = []
    top = []
    bottom = []
    for index, row in euclidean.iterrows(): # Traverse through all the words in the paragraph
      box_id = int(row['Id'])
      if box_id in i[0]:
        # To find the coordinates of the top-left and bottom-right for the paragraph to draw the paragraph bounding boxes.
        # Find the minimum coordinate for the left and top coordinate, and maximum coordinate for the right and bottom coordinate.
        right_box_string = row['Right']
        left_box_string = row['Left']
        top_box_string = row['Top']
        bottom_box_string = row['Bottom']
        right_box = parse_string(right_box_string,"[",",")
        left_box = parse_string(left_box_string,"[",",")
        top_box = parse_string(top_box_string,",","]")
        bottom_box = parse_string(bottom_box_string,",","]")
        if(int(round(float(left_box)))!=-1):
          left.append(int(round(float(left_box))))
        if(int(round(float(right_box)))!=-1):
          right.append(int(round(float(right_box))))
        if(int(round(float(top_box)))!=-1):
          top.append(int(round(float(top_box))))
        if(int(round(float(bottom_box)))!=-1):
          bottom.append(int(round(float(bottom_box))))
    l = min(left)
    r = max(right)
    t = min(top)
    b = max(bottom)
    center_top = [int(l+r)/2, int(t)]
    center_bottom = [int(l+r)/2, int(b)]
    center_right = [int(r), int(t+b)/2]
    center_left = [int(l), int(t+b)/2]
    larger_box_top_left = (int(l - 20), int(t - 20))
    larger_box_bottom_right = (int(r + 10), int(b + 10))
    bottom_box = [-1, 0]
    visited = 0
    order = -1
    # Uncomment to generate the image
    # cv2.rectangle(image_with_boxes, larger_box_top_left, larger_box_bottom_right, (0, 0, 255), 2)
    new_row = pd.Series([i, count, center_top, center_bottom, center_right, center_left, bottom_box, visited, order])
    component = component.append(new_row, ignore_index=True)
    count = count+1
#  uncomment to show the image and write it to an output file
#  plt.imshow(image_with_boxes)
#  plt.axis('off')
#  plt.show()
#  output_path = 'Para.png'
#  cv2.imwrite(output_path, cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))
 new_column_names = {0: 'Component', 1: 'Id', 2: 'Top',3: 'Bottom',4: 'Right',5: 'Left',6: 'Bottom_Box',7: 'Visited',8: 'Order'}
 component = component.rename(columns=new_column_names)
 component = ignore_margins(component, 20, 0, image_filename)
 component = component.reset_index(drop=True)
 calculate_bottombox_para(component)
 return component

In [ ]:
# Function to find the minimum euclidean distance from the top-left corner of the page to the top-left corner of the paragraph bounding box.
# Input: Dataframe containing paragraph information
# Output: Paragraph id of the closest paragraph to the top-left corner of the page.
def minimum_euclidean(component):
    euclidean = float('inf')
    min_idx = -1 # Initialise it to -1
    for i in range(len(component)):
        if component['Visited'][i] != 1:
            current_distance = euclidean_distance(np.array([0, 0]), np.array(component['Top'][i]))
            if current_distance < euclidean:
                euclidean = current_distance
                min_idx = i
    return min_idx

In [ ]:
# Function to connect current paragraph with the paragraph immediately below it if it exists.
# Input: Dataframe with paragraph information
# Output: Updated Dataframe with Bottom box
def calculate_bottombox_para(new_df):
    bottom = []
    for i in range(len(new_df)): # Traverse all the components
        dist = []
        id = []
        for j in range(len(new_df)): # For each component look for closest paragraphs on the bottom
            distance = euclidean_distance(np.array(new_df['Top'][j]), np.array(new_df['Bottom'][i]))
            # similar to making connections with words
            if 0 < distance < 60 and i != j: # 60 can be replaced with a KDE function similar to word thresholds
                dist.append(distance)
                id.append(j)
        if dist:
            t = np.argmin(dist)
            bottom.append([np.min(dist), id[t]])
        else:
            bottom.append([-1, 0]) # Default value

    new_df['Bottom_Box'] = bottom

In [ ]:
# Function to get the bottom paragraph if connected
# Input: Dataframe with paragraph information, and current paragraph id
# Output: Id of the bottom paragraph
def get_next(component, i):
  if(int(component['Bottom_Box'][i][0]) == -1):
    return -1
  else:
    return int(component['Bottom_Box'][i][1])

In [ ]:
# Function to generate paragraph order
# Input: Dataframe with paragraph information
# Output: Updated Dataframe with Paragraph order
def paragraph_order(component):
    order = 0
    min_idx = minimum_euclidean(component) # Get the closest paragraph id

    while any(component['Visited'] == 0) and min_idx != -1:
        if component['Visited'][min_idx] != 1:
            component['Visited'][min_idx] = 1
            component['Order'][min_idx] = order
            order += 1

        next_idx = get_next(component, min_idx)
        if next_idx != -1:
            min_idx = next_idx
        else:
            min_idx = minimum_euclidean(component)

    return component

In [ ]:
# Function to visualise paragraph order
# Input: image, target components, dataframe with word bounding boxes, dataframe with paragraph bounding boxes
def visualise_paragraph_order(image, target_components, euclidean,component):

 image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 count = 0
 image_with_boxes = image_rgb.copy()
 for i in target_components:
    left1 = []
    right1 = []
    top1 = []
    bottom1 = []
    for index, row in euclidean.iterrows():
      box_id = int(row['Id'])
      if box_id in i[0]:
        right_box1 = row['Right']
        left_box1 = row['Left']
        top_box1 = row['Top']
        bottom_box1 = row['Bottom']
        right_box = parse_string(right_box1,"[",",")
        left_box = parse_string(left_box1,"[",",")
        top_box = parse_string(top_box1,",","]")
        bottom_box = parse_string(bottom_box1,",","]")
        if(int(round(float(left_box)))!=-1):
          left1.append(int(round(float(left_box))))
        if(int(round(float(right_box)))!=-1):
          right1.append(int(round(float(right_box))))
        if(int(round(float(top_box)))!=-1):
          top1.append(int(round(float(top_box))))
        if(int(round(float(bottom_box)))!=-1):
          bottom1.append(int(round(float(bottom_box))))
    l = min(left1)
    r = max(right1)
    t = min(top1)
    b = max(bottom1)
    center_top = [int(l+r)/2, int(t)]
    center_bottom = [int(l+r)/2, int(b)]
    center_right = [int(r), int(t+b)/2]
    center_left = [int(l), int(t+b)/2]
    larger_box_top_left = (int(l - 20), int(t - 20))
    larger_box_bottom_right = (int(r + 10), int(b + 10))

    cv2.rectangle(image_with_boxes, larger_box_top_left, larger_box_bottom_right, (0, 0, 255), 2)
    cv2.putText(image_with_boxes, str(component['Order'][count]), (int(l - 20), int(t - 30)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    count = count + 1
 # Write to output path
 output_path = 'paragraph_order.png'
 cv2.imwrite(output_path, cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))

## Stage-4: The word bounding box sequence is obtained for each paragraph. This, combined with the sequential order of paragraphs from Stage-3, establishes the reading order for words in the document.

In [ ]:
# Function to return id of the box to the right if a connection exists
# Input: Dataframe with bounding box information
# Output: ID of box to the immediate right if connection exists
def get_next_word(euclidean, i):
  if(int(float(parse_string(euclidean['Right_Box'][i],'[',','))) == -1):
    return -1 # Default return value
  else:
    return int(float(parse_string(euclidean['Right_Box'][i],',',']')))


In [ ]:
# Function to get the next line
# Input: Dataframe with paragraph information, dataframe with bounding box information, paragraph id
# Output: bounding box id from the immediate next line
def minimum_distance(component, euclidean, i):
  # Calculate the minimum x and y coordinates to the top left corner of the paragraph bounding box and returns the id that is closest and not visited.
    # initialise values
    min_x_distance = math.inf
    min_y_distance = math.inf
    closest_coordinate = -1

    for j in component['Component'][i][0]:
        index = euclidean.index[euclidean['Id'] == j]
        if euclidean['Visited'].loc[index].values[0] != 1:
            y_distance = abs(0 - float(parse_string(euclidean['Top'].loc[index].values[0], ',', ']')))
            x_distance = abs(0 - float(parse_string(euclidean['Left'].loc[index].values[0], '[', ',')))

            if (y_distance < min_y_distance or (y_distance == min_y_distance and x_distance < min_x_distance)) and int(float(parse_string(euclidean['Left_Box'].loc[index].values[0],'[',','))) == -1:
                min_x_distance = x_distance
                min_y_distance = y_distance
                closest_coordinate = j # Id of the closest box

    return closest_coordinate

In [ ]:
# Function to caluclate the box immediately right to current box if connection does not exist
# Input: Dataframe with paragraph information, current box id, dataframe with bounding box information, paragraph id
# Output: bounding box id from the immediate next right
def calculate_next_right(component, min_idx, euclidean, i):
  # Calculate the minimum x and y coordinates to the right bounding box and returns the id that is closest and not visited.
  # initialise values
    min_x_distance = math.inf
    min_y_distance = math.inf
    closest_coordinate = -1
    index_min = euclidean.index[euclidean['Id'] == min_idx].tolist()[0]
    component_list = component['Component'][i][0]
    for j in component_list:
        index = euclidean.index[euclidean['Id'] == j].tolist()[0]
        if euclidean.at[index, 'Visited'] != 1:
            x_distance = abs(float(parse_string(euclidean.at[index_min, 'Right'],'[',',')) - float(parse_string(euclidean.at[index, 'Left'],'[',',')))
            y_distance = abs(float(parse_string(euclidean.at[index_min, 'Right'],',',']')) - float(parse_string(euclidean.at[index, 'Left'],',',']')))
            if x_distance < min_x_distance and y_distance < 10:
              min_x_distance = x_distance
              min_y_distance = y_distance
              closest_coordinate = j # Id of the closest box
    return closest_coordinate

In [ ]:
# Function to generate reading order
# Input: Dataframe with paragraph information, dataframe with bounding box information
# Output: Updated dataframe with reading order
def word_order(component, euclidean):
  # initialise values
    euclidean.loc[:, 'Order'] = -1
    euclidean.loc[:, 'Visited'] = 0
    order = 0
    for i in range(len(component)):
        min_idx = minimum_distance(component, euclidean, i)
        visited_list = [euclidean['Visited'][idx] == 0 for idx in component['Component'][i][0]]
        while any(visited_list) != 0 and min_idx != -1:
            if euclidean.at[min_idx, 'Visited'] != 1:
                euclidean.at[min_idx, 'Visited'] = 1
                euclidean.at[min_idx, 'Order'] = order
                order += 1
            next_idx = get_next_word(euclidean, min_idx)
            if next_idx != -1:
                min_idx = next_idx
            elif calculate_next_right(component, min_idx, euclidean, i) != -1:
                min_idx = calculate_next_right(component, min_idx, euclidean, i)
            else:
                min_idx = minimum_distance(component, euclidean, i)

    return euclidean

In [ ]:
# Function to visualise reading order
# Input: Image, dataframe with bounding box information
# Output: image generated with reading order
def reading_order(image,euclidean):

  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_with_boxes = image_rgb.copy()
  for index, row in euclidean.iterrows():
      left = int(parse_string(row['Left'],'[',','))
      right = int(parse_string(row['Right'],'[',','))
      top = int(parse_string(row['Top'],',',']'))
      bottom = int(parse_string(row['Bottom'],',',']'))
      Order = int(row['Order'])

      width = right - left
      height = bottom - top

      top_left = (left, top)
      bottom_right = (right, bottom)
      # Uncomment to show boxes
      # cv2.rectangle(image_with_boxes, top_left, bottom_right, (255, 0, 0), 2)

      label_position = (left, top - 10)
      cv2.putText(image_with_boxes, str(Order), label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

  return image_with_boxes

### Ignore Header and Footer

In [ ]:
# Function to return size of the page
def page_size(image_file):
  image = cv2.imread(image_file)
  height, width, _ = image.shape
  return height, width

In [ ]:
# Function to remove all the components present within the margin
# Input: Dataframe with paragraph information, % of the height to be ignored, % of the width to be ignored, image
def ignore_margins(component, height_p, width_p, image_file):
  height, width = page_size(image_file)
  vertical_margin = height*(height_p/100)
  horizontal_margin = width*(width_p/100)
  for i in range(len(component)):
    if((component['Top'][i][1] > height - vertical_margin) and len(component['Component'][i][0])<7):
      component = component.drop(i)
    elif((component['Bottom'][i][1] < vertical_margin) and len(component['Component'][i][0])<7):
      component = component.drop(i)
    elif(component['Right'][i][0] < horizontal_margin):
      component = component.drop(i)
    elif(component['Left'][i][0] > width - horizontal_margin):
      component = component.drop(i)
    else:
      continue
  return component

## Reading Order Generator Function

In [ ]:
def Reading_Order_Generator(image_file, doctr_file):

  df = pd.read_json(doctr_file)
  df = df.T
  euclidean = pd.DataFrame()
  kde = pd.DataFrame()
  calculate_center_points(df, kde)
  horizontal_neighbors, vertical_neighbors = find_closest_neighbors(kde)
  x = kde_estimate(horizontal_neighbors)
  y = kde_estimate(vertical_neighbors)
  calculate_center_points(df,euclidean)
  calculate_rightbox(euclidean,x)
  calculate_leftbox(euclidean,x)
  calculate_topbox(euclidean,y)
  calculate_bottombox(euclidean,y)
  euclidean.to_csv("connections.csv")
  euclidean = pd.read_csv("connections.csv")
  G = create_graphs(euclidean)
  get_paras(G)
  component = pd.DataFrame()
  euclidean = pd.read_csv('connections.csv')
  d1 = pd.read_json('paragraph.json')
  target_components = d1.values.tolist()
  image = cv2.imread(image_file)
  component = recognise_paragraphs(image, target_components, euclidean, image_file)
  min_idx =  minimum_euclidean(component)
  component = paragraph_order(component)
  # Uncomment this code to visualise paragraph order
  # visualise_paragraph_order(image, target_components, euclidean,component)
  new = pd.DataFrame()
  sort_order = component.sort_values('Order').index
  component.to_csv('Component.csv')
  new = component.loc[sort_order]
  new = new.reset_index(drop=True)
  new_euclidean = pd.DataFrame()
  euclidean = word_order(new, euclidean)
  sort_order = euclidean.sort_values('Order').index
  new_euclidean = euclidean.loc[sort_order]
  new_euclidean = new_euclidean.reset_index(drop=True)
  image = cv2.imread(image_file)
  image_with_boxes = reading_order(image,new_euclidean)
  output_path = 'Output.png'
  cv2.imwrite(output_path, cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))
  euclidean.to_csv('Euclidean.csv')

In [ ]:
image_file = 'Path_to_image' # Add the path to the image

pred = doctr_predictions(image_file)

data = {}

json_data = json.dumps(pred)
with open("doctr_file.json", "w") as outfile:
    outfile.write(json_data)

doctr_file = '/content/doctr_file.json' # Doctr predicctions stored as doctr_file.json
Reading_Order_Generator(image_file, doctr_file)